# Download Data

In [8]:
!pip install -q kaggle
import json

#Create Credential
######################## Insert Your username and Key ######################
data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
######################## Insert Your username and Key ######################
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

In [9]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c brain-motor-imagery-classification

ref                                                             title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rahulvyasm/netflix-movies-and-tv-shows                          Netflix Movies and TV Shows                        1MB  2024-04-10 09:48:38          17826        388  1.0              
sahirmaharajj/school-student-daily-attendance                   School Student Daily Attendance                    2MB  2024-04-29 19:29:56           4006         87  1.0              
ashishkumarak/amazon-shopping-reviews-daily-updated             Amazon Shopping Reviews [Daily Updated]            7MB  2024-05-13 23:35:11            790         25  1.0              
supriyoain/water-quality-data                                   Water Quali

In [10]:
!unzip /content/brain-motor-imagery-classification.zip

Archive:  /content/brain-motor-imagery-classification.zip
  inflating: sample_submission.csv   
  inflating: test/00edaaa8-102d-4ee3-b531-0be67dcbd9e6.npy  
  inflating: test/01b63df8-05f9-4956-87f4-949ed9ae4dd6.npy  
  inflating: test/01bfd881-7850-49f1-ad32-ff9ae51e3e3f.npy  
  inflating: test/01d9002f-b086-4635-bf74-5316ceda3098.npy  
  inflating: test/028522a4-5332-4e84-b578-ce45ed5e4899.npy  
  inflating: test/0368a7cc-d6c6-416d-878e-edbcd562f317.npy  
  inflating: test/044841bc-050d-4c9f-92db-1ff1a7f306cb.npy  
  inflating: test/051581bf-c779-4d9c-86e9-324404b149ca.npy  
  inflating: test/06360991-5140-41a1-889f-3cb10b2241d0.npy  
  inflating: test/066dc317-8b3c-431e-b5d9-894abe3a4857.npy  
  inflating: test/06e9b7ab-0ee9-47a3-a500-a8dab4ee097f.npy  
  inflating: test/06f9ab25-f9b7-4a12-82f3-a3c92adcc645.npy  
  inflating: test/072dc9a9-7bfa-4da9-83d7-23b2ed5818e0.npy  
  inflating: test/075e9017-dace-441d-acbd-dd929c1a06f7.npy  
  inflating: test/0783e1ea-375e-4239-be7d-12177a54

<hr>

# Prepare Data

In [14]:
from scipy.signal import butter, filtfilt, iirnotch, freqz

In [12]:
# Band-pass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# Notch filter
def notch_filter(data, cut_freq, fs, Q=30):
    nyq = 0.5 * fs
    freq = cut_freq / nyq
    b, a = iirnotch(freq, Q)
    y = filtfilt(b, a, data)
    return y

def filter03050(Ipsignal):
    sampling_rate = 250  # Hz
    # fft_signal = np.fft.fft(Ipsignal)
    # frequencies = np.fft.fftfreq(len(Ipsignal), 1/sampling_rate)
    filtered_signal = bandpass_filter(Ipsignal, 1, 30, sampling_rate)          # band-pass filter 0-30 Hz
    filtered_signal = notch_filter(filtered_signal, 50, sampling_rate)       # notch filter 50 Hz
    return filtered_signal

In [15]:
file_path = "/content/train/train/"
from torch.utils.data import Dataset
class SignalDataset(Dataset):
    def __init__(self,Datapath,filter_function=None):
        self.filtered_fn = filter_function
        self.Datapath = Datapath

        self.unique_name = self.unique_name()
        self.dataset  = self.openfile()
        # print(self.dataset)

        self.signals = []
        self.labels = []
        for segments in self.dataset:
            segment, label = segments
            self.signals.append(segment)
            self.labels.append(label)

    def unique_name(self):
        all_name = os.listdir(self.Datapath)
        new = []
        for un in all_name:
            i = un.replace("_data_time_series.npy","").replace("_data_time_stamps.npy","").replace("_label_time_series.npy","").replace("_label_time_stamps.npy","")
            new.append(i)
            unique_name = list(set(new))
        return unique_name

    # Check Diff time function
    def check_different_duration(self,label_time_stamps):
        keep_time = []
        start_time = 0
        for current_time in list(label_time_stamps):
            keep_time.append(current_time-start_time)
            start_time = current_time
        keep_time = keep_time[1:]
        self.avgtime = sum(keep_time)/len(keep_time)
        stat = True
        for etime in keep_time:
            if 6.8 <= etime <= 7.5:
                stat = True
            else:
                stat = False
                break
        return stat

    def openfile(self):
        dataset = {ds:[] for ds in self.unique_name}
        for name in self.unique_name:
            signals           = np.load(os.path.join(file_path,name+"_data_time_series.npy"))
            times             = np.load(os.path.join(file_path,name+"_data_time_stamps.npy"))
            label_time_series = np.load(os.path.join(file_path,name+"_label_time_series.npy"))
            label_time_stamps = np.load(os.path.join(file_path,name+"_label_time_stamps.npy"))
            if not(self.check_different_duration(label_time_stamps)):
                continue
            # print("Avg time",self.avgtime)
            if filter!=None:
                filtered_signal = []
                Dim = signals.shape[1]
                for i in range(Dim):
                    # print(f"{i} : {signals[:,i]}")
                    filtered_signal.append(self.filtered_fn(signals[:,i]))
                signals = np.array(filtered_signal).T
            # dataset[name] = [signals,times,label_time_series,label_time_stamps,int(round(self.avgtime))]
            slices = []
            sample_point = int(round(self.avgtime))
            datapoint = 250 * sample_point                   # 250 Hz * 7 s
            for index in range(30):
                start_point = index * datapoint
                end_point = (index + 1) * datapoint
                if end_point <= signals.shape[0]:  # Check to ensure slice is within the data range
                    slices.append(signals[start_point:end_point])
                else:
                    break  # Break if we run out of data points to slice
                # slices.append(signals[slice * 1750: (slice + 1) * 1750])
                # signals[slice * datapoint: (slice + 1) * datapoint]
            # return tuple(zip(slices, label_time_series))
            data_sliced_with_label = tuple(zip(slices,label_time_series))
            return data_sliced_with_label
        #     dataset[name] = data_sliced_with_label
        # return dataset

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        return self.signals[idx], self.labels[idx]


Yp = SignalDataset("/content/train/train",filter_function = filter03050)

In [17]:
Yp[0]

(array([[-1.17996358e+01, -2.51361191e+00, -5.28005002e+00, ...,
         -3.51569073e-08, -6.90616842e-01, -4.57533312e-10],
        [-7.42296340e+00,  1.26065058e+01, -3.75898394e+00, ...,
         -3.36909366e-08, -4.48998567e-01, -4.37853362e-10],
        [-4.50066454e+00,  2.38360727e+01, -2.88879562e+00, ...,
         -3.22348849e-08, -2.17347926e-01, -4.18335757e-10],
        ...,
        [-7.44688213e-01, -1.26183943e+01, -9.91829934e+00, ...,
         -5.24079895e-09, -1.90913839e-05,  1.57126482e-11],
        [-1.06077578e+00, -1.24898007e+01, -9.04034080e+00, ...,
         -5.49612983e-09, -1.89723385e-05,  1.66815210e-11],
        [-2.83064078e+00, -1.33447571e+01, -9.47502492e+00, ...,
         -5.74650003e-09, -1.88418795e-05,  1.76330433e-11]]),
 array([120], dtype=int32))

<hr>

# Pipeline